Utilizando regresión logística, clasificá a los países del conjunto de datos de expectativa de vida de la práctica 2 en base a la columna Status. Evaluá si hay un desbalance de clases, y cómo impacta en la clasificación. Indicá, en base a los pesos encontrados del modelo, a qué atributos se le da mayor importancia para clasificar a los ejemplos.

In [77]:
%load_ext autoreload
%autoreload 2
%matplotlib 
import matplotlib.pyplot as plt
import utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Using matplotlib backend: Qt5Agg


In [78]:
import tensorflow as tf
import keras
import numpy as np
import utils 
import os
import pandas as pd
import sklearn

from sklearn import preprocessing

In [79]:
# Carga del dataset
data=pd.read_csv('../Practica 5/who_life_expectancy.csv')

In [80]:
data.head()

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,Afghanistan,2015,Developing,65.0,263.0,62,0.01,71.279624,65.0,1154,...,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,Afghanistan,2014,Developing,59.9,271.0,64,0.01,73.523582,62.0,492,...,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,Afghanistan,2013,Developing,59.9,268.0,66,0.01,73.219243,64.0,430,...,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,Afghanistan,2012,Developing,59.5,272.0,69,0.01,78.184215,67.0,2787,...,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,Afghanistan,2011,Developing,59.2,275.0,71,0.01,7.097109,68.0,3013,...,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5


In [81]:
# Paso a numericos las variables categoricas
data['Status'] = pd.factorize(data['Status'])[0]

# Se elimina el pais
data = data.drop(['Country'], 1)

In [82]:
data.isnull().values.any()
# Elimino filas con faltantes
data = data.dropna()

In [83]:
# Se normalizan las variables menos la de Status que es la variable y
x = data.loc[:, data.columns != 'Status'].values #returns a numpy array

min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)

y = data.loc[:, 'Status'].values

In [84]:
# cantidad de ejemplos y dimension de entrada
n,d_in=x_scaled.shape
# calcula la cantidad de clases
classes=int(y.max())+1

print("Información del conjunto de datos:")
print(f"Ejemplos: {n}")
print(f"Variables de entrada: {d_in}")
print(f"Cantidad de clases: {classes}")

Información del conjunto de datos:
Ejemplos: 1649
Variables de entrada: 20
Cantidad de clases: 2


In [85]:
# Veo si hay desbalance de clases
data['Status'].value_counts()

0    1407
1     242
Name: Status, dtype: int64

In [86]:
# Creación del modelo inicial
print("Inicialización aleatoria del modelo (podes volver a correr esta celda para obtener otros resultados)")

# Creo un modelo logístico
modelo = keras.Sequential([
    # la activación softmax hace que la salida sean probabilidades
    keras.layers.Dense(classes,input_shape=(d_in,), activation='softmax')])

modelo.compile(
  optimizer=keras.optimizers.SGD(lr=0.001), 
  loss='sparse_categorical_crossentropy', 
  # metricas para ir calculando en cada iteracion o batch 
  # Agregamos el accuracy del modelo
  metrics=['accuracy'], 
)

# Entrenamiento del modelo
modelo.fit(x_scaled,y,epochs=100,batch_size=16)


Inicialización aleatoria del modelo (podes volver a correr esta celda para obtener otros resultados)
Epoch 1/100
1649/1649 [==============================] - 0s 126us/step - loss: 0.7848 - acc: 0.2711
Epoch 2/100
1649/1649 [==============================] - 0s 47us/step - loss: 0.6592 - acc: 0.6598
Epoch 3/100
1649/1649 [==============================] - 0s 45us/step - loss: 0.5835 - acc: 0.7914
Epoch 4/100
1649/1649 [==============================] - 0s 43us/step - loss: 0.5369 - acc: 0.8351
Epoch 5/100
1649/1649 [==============================] - 0s 42us/step - loss: 0.5074 - acc: 0.8466
Epoch 6/100
1649/1649 [==============================] - 0s 45us/step - loss: 0.4881 - acc: 0.8490
Epoch 7/100
1649/1649 [==============================] - 0s 45us/step - loss: 0.4750 - acc: 0.8502
Epoch 8/100
1649/1649 [==============================] - 0s 44us/step - loss: 0.4657 - acc: 0.8526
Epoch 9/100
1649/1649 [==============================] - 0s 47us/step - loss: 0.4590 - acc: 0.8532
Epoch 1

In [87]:
# Obtencion de valores de w y b
w,b=modelo.get_weights()
print(f"w: {w}, b: {b}")

w: [[ 0.17990884 -0.21935108]
 [-0.03874977 -0.24477923]
 [ 0.11696236  0.1937641 ]
 [ 0.49989653  0.34398568]
 [-0.8334688   0.19583681]
 [ 0.04512671 -0.3684747 ]
 [ 0.0763887   0.21355098]
 [ 0.07095677  0.36849204]
 [ 0.0555912   0.2482347 ]
 [-0.23747896 -0.18990548]
 [ 0.42507863  0.01729322]
 [-0.03046526 -0.47634822]
 [ 0.5920441  -0.37355113]
 [ 0.02148602 -0.36084813]
 [-0.09752234  0.31125382]
 [ 0.12303178  0.38607556]
 [ 0.3487622  -0.25530756]
 [-0.27906603 -0.7341225 ]
 [-0.29912844  0.32099396]
 [-0.07810958  0.3769105 ]], b: [ 0.47267562 -0.47267562]


Le da mayor importancia a la variable 3 que es el life expectancy. Como esta muy desbalanceada la clase da bien porque si predice la variable 0 es muy porbable que sea correcto. Se debe hacer subsampleo o usar algun metodo para aumentar la otra clase.